## 🤗 Finetune **Longformer Encoder-Decoder (LED)** for Abstract Generation 🤗


---
This notebook is based on the training script provided with the [LED](https://huggingface.co/transformers/model_doc/led.html) model from the [Huggingface Transformers](https://huggingface.co/transformers/) library. The original script can be found [here](https://colab.research.google.com/drive/12LjJazBl7Gam0XBPy_y0CTOJZeZ34c2v?usp=sharing#scrollTo=6GRz0rksYb3h)


---
The *Longformer Encoder-Decoder (LED)* was recently added as an extension to [Longformer: The Long-Document Transformer](https://arxiv.org/abs/2004.05150) by Iz Beltagy, Matthew E. Peters, Arman Cohan.

In this notebook we will finetune *LED* for Summarization on [Pubmed](https://huggingface.co/datasets/viewer/?dataset=scientific_papers). *Pubmed* is a long-range summarization dataset, which makes it a good candidate for LED. LED will be finetuned up to an input length of 8K tokens on a single GPU.

We will leverage 🤗`Seq2SeqTrainer`, gradient checkpointing and as usual 🤗`datasets`.

Training this model takes a decently powerful GPU. The original notebook recommends a GPU with at least 15GB of VRAM. Fortunately, we have access to cloud computing resources, so we are able to do run experiments with thos model.

In [1]:
!nvidia-smi

Sat Nov 26 20:36:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    54W / 400W |      0MiB / 40960MiB |     27%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Install all of the packages needed for this project. We need to use the `-f https://download.pytorch.org/whl/torch_stable.html` flag to install the correct version of PyTorch for the GPU we are using.

In [2]:
!pip install -r requirements.txt -f https://download.pytorch.org/whl/torch_stable.html

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


## Dataset

Let's start by loading and preprocessing the dataset. NOTE: we will have to change this slightly when we switch to predicting the introduction instead of the abstract.

In [3]:
from datasets import load_dataset, load_metric

In [4]:
def lists_to_single_str(dataset):
    dataset['introduction'] = dataset['section_texts'][0]
    dataset['section_titles'] = '\n'.join(dataset['section_titles'])
    dataset['section_texts'] = '\n'.join([dataset['abstract']] + dataset['section_texts'][1:]) # add abstract to section_texts and remove introduction

    return dataset

# load the dataset from cnn_papers.json and nlp_papers.json (TODO: include ml_papers.json)
dataset = load_dataset('json', data_files=['cnn_papers.json', 'nlp_papers.json'], split='train')
dataset = dataset.map(lists_to_single_str) # convert the paper sections into a format that can be processed by the model

Using custom data configuration default-77495a8d85b716fa
Found cached dataset json (/home/jupyter/.cache/huggingface/datasets/json/default-77495a8d85b716fa/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
Loading cached processed dataset at /home/jupyter/.cache/huggingface/datasets/json/default-77495a8d85b716fa/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-810cb1c860ca3f13.arrow


Right now, the dataset should be split with an 80/20 train/test split. We may change this later to a train/val/test split.

In [5]:
seed = 42 # set the seed for reproducibility (and so that we can get the same test dataset when evaluating the model)

dataset = dataset.train_test_split(test_size=0.2, seed=seed)
print(len(dataset['train']))
print(len(dataset['test']))

Loading cached split indices for dataset at /home/jupyter/.cache/huggingface/datasets/json/default-77495a8d85b716fa/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-93e92c9e92be8bcb.arrow and /home/jupyter/.cache/huggingface/datasets/json/default-77495a8d85b716fa/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-c7a4c20f0f95f527.arrow


78
20


Let's take a quick look at one of the papers

In [6]:
import random

paper = dataset['train'][random.randint(0, len(dataset['train']))]
print(paper['title'], '\n')
print(paper['section_titles'], '\n')
print(paper['introduction'])

Analytical Techniques for Developing Argumentative Writing in STEM: A Pilot Study 

Introduction
Background
Methodology
Discussion
Conclusion 

Good writing, especially making effective arguments, demonstrates excellent critical thinking skills [1]. Writing as a means of communicating knowledge is a necessity in higher education (HE). Yet students enrolled in STEM programs worldwide often have little opportunity to develop and practice writing during their college years. Several studies have shown that graduates in computer science and engineering seldom have adequate writing skills for work in a professional setting [2], [3]. In fact, the problem starts earlier. As Gibbs [4] argues, many students in the U.K. leave secondary school without proficiency in reading, writing, and communication. In the U.S., student achievement in reading and writing from the National Assessment of Educational Progress points to a serious crisis in writing instruction: most students do not achieve grade pro

## Tokenizing

Now, we tokenize it using a Autotokenizer from HuggingFace.

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

Note that for the sake of this notebook, we finetune the "smaller" LED checkpoint ["allenai/led-base-16384"](https://huggingface.co/allenai/led-base-16384). Better performance can however be attained by finetuning ["allenai/led-large-16384"](https://huggingface.co/allenai/led-large-16384) at the cost of a higher required GPU RAM.

In [8]:
max_input_length = 16384
max_output_length = 1024
batch_size = 1 # this is the largest batch size we can fit even with an A100 40 GB GPU

Now, let's write down the input data processing function that will be used to map each data sample to the correct model format.
As explained earlier `section_texts` represents here our input data and `introduction` is the target data. The max input length is set to 16384 because that is long enough to handle every paper. The output length is 2048 is the smallest multiple of 2 that covers the entire introduction dataset.

In addition to the usual `attention_mask`, LED can make use of an additional `global_attention_mask` defining which input tokens are attended globally and which are attended only locally, just as it's the case of [Longformer](https://huggingface.co/transformers/model_doc/longformer.html). For more information on Longformer's self-attention, please take a look at the corresponding [docs](https://huggingface.co/transformers/model_doc/longformer.html#longformer-self-attention). For summarization, we follow recommendations of the [paper](https://arxiv.org/abs/2004.05150) and use global attention only for the very first token. Finally, we make sure that no loss is computed on padded tokens by setting their index to `-100`.

In [9]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["section_texts"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["introduction"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

Now that we have a function that tokenizes the data, we can apply it to our dataset.

In [10]:
dataset = dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["title", "section_texts", "abstract", "introduction", "section_titles"], # remove the columns that we don't need anymore since we've already tokenized them
)

  0%|          | 0/78 [00:00<?, ?ba/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

Finally, the datasets should be converted into the PyTorch format as follows.

In [11]:
dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

## Model

Alright, we're almost ready to start training. Let's load the model via the `AutoModelForSeq2SeqLM` class.

In [12]:
from transformers import AutoModelForSeq2SeqLM

We've decided to stick to the smaller model `"allenai/led-base-16384"` for the sake of this notebook. In addition, we directly enable gradient checkpointing and disable the caching mechanism to save memory.

In [13]:
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

During training, we want to evaluate the model on Rouge, the most common metric used in summarization, to make sure the model is indeed improving during training. For this, we set fitting generation parameters. We'll use beam search with a small beam of just 2 to save memory. Also, we force the model to generate at least 100 tokens, but no more than 512. In addition, some other generation parameters are set that have been found helpful for generation. For more information on those parameters, please take a look at the [docs](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate).

In [14]:
# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = max_output_length
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

Next, we also have to define the function the will compute the `"rouge"` score during evalution.

Let's load the `"rouge"` metric from 🤗datasets and define the `compute_metrics(...)` function.

In [15]:
rouge = load_metric("rouge")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


The compute metrics function expects the generation output, called `pred.predictions` as well as the gold label, called `pred.label_ids`.

Those tokens are decoded and consequently, the rouge score can be computed.

In [16]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

## Training

Now, we're ready to start training. Let's import the `Seq2SeqTrainer` and `Seq2SeqTrainingArguments`.

In [17]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In contrast to the usual `Trainer`, the `Seq2SeqTrainer` makes it possible to use the `generate()` function during evaluation. This should be enabled with `predict_with_generate=True`. Because our GPU RAM is limited, we make use of gradient accumulation by setting `gradient_accumulation_steps=4` to have an effective `batch_size` of 2 * 4 = 8.

Other training arguments can be read upon in the [docs](https://huggingface.co/transformers/main_classes/trainer.html?highlight=trainingarguments#transformers.TrainingArguments).

In [18]:
# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="./",
    gradient_accumulation_steps=4,
    num_train_epochs=5, # since our dataset is so small, we may want to train for more epochs
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The training arguments, along with the model, tokenizer, datasets and the `compute_metrics` function can then be passed to the `Seq2SeqTrainer`

In [19]:
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
)

Using cuda_amp half precision backend


Now we can start training.

In [20]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 78
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 95
You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
0,3.573500,3.329346,0.098200,0.108100,0.101200
1,3.161800,3.242111,0.108400,0.100800,0.102300
2,2.957400,3.219178,0.126200,0.110300,0.115300
3,2.826500,3.205578,0.140800,0.116400,0.122400
4,2.668100,3.217054,0.134100,0.123000,0.124900


***** Running Evaluation *****
  Num examples = 20
  Batch size = 1
***** Running Evaluation *****
  Num examples = 20
  Batch size = 1
***** Running Evaluation *****
  Num examples = 20
  Batch size = 1
***** Running Evaluation *****
  Num examples = 20
  Batch size = 1
***** Running Evaluation *****
  Num examples = 20
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=95, training_loss=3.0374575163188733, metrics={'train_runtime': 4483.9258, 'train_samples_per_second': 0.087, 'train_steps_per_second': 0.021, 'total_flos': 4190717903634432.0, 'train_loss': 3.0374575163188733, 'epoch': 4.97})

## Evaluation

Lets take a look at how well the original model compares to the fine-tuned model.

First, lets load the test dataset.

In [21]:
from datasets import load_dataset

def lists_to_single_str(dataset):
    dataset['introduction'] = dataset['section_texts'][0]
    dataset['section_titles'] = '\n'.join(dataset['section_titles'])
    dataset['section_texts'] = '\n'.join([dataset['abstract']] + dataset['section_texts'][1:]) # add abstract to section_texts and remove introduction

    return dataset

# load the dataset from cnn_papers.json and nlp_papers.json (TODO: include ml_papers.json)
dataset = load_dataset('json', data_files=['cnn_papers.json', 'nlp_papers.json'], split='train')
dataset = dataset.map(lists_to_single_str) # convert the paper sections into a format that can be processed by the model
seed = 42 # set the seed for reproducibility

dataset = dataset.train_test_split(test_size=0.2, seed=seed)

And we can define a function that will let us evaluate the model on the test dataset.

In [22]:
import torch
import copy
from datasets import load_metric

def evaluate_model(model, tokenizer):
    model.eval()

    def generate_answer(batch):
      inputs_dict = tokenizer(batch["section_texts"], padding="max_length", max_length=max_input_length, return_tensors="pt", truncation=True)
      input_ids = inputs_dict.input_ids.to("cuda")
      attention_mask = inputs_dict.attention_mask.to("cuda")
      global_attention_mask = torch.zeros_like(attention_mask)
      # put global attention on <s> token
      global_attention_mask[:, 0] = 1

      predicted_abstract_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
      batch["predicted_introduction"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)

      return batch

    result = dataset['test'].map(generate_answer, batched=True, batch_size=4)

    # load rouge
    rouge = load_metric("rouge")

    print("Rouge Results:", rouge.compute(predictions=result["predicted_introduction"], references=result["introduction"], rouge_types=["rouge2"])["rouge2"].mid)

    return result

Here are the results from the original LED-Base-16384 model.

In [23]:
from transformers import AutoTokenizer

original_tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")
original_model = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384").to('cuda')

# set generate hyperparameters to be same as LED
original_model.config.num_beams = 2
original_model.config.max_length = max_output_length
original_model.config.min_length = 100
original_model.config.length_penalty = 2.0
original_model.config.early_stopping = True
original_model.config.no_repeat_ngram_size = 3

loading configuration file config.json from cache at /home/jupyter/.cache/huggingface/hub/models--allenai--led-base-16384/snapshots/25756ed025a94fdf2bc4987af86a58fd999047ec/config.json
Model config LEDConfig {
  "_name_or_path": "allenai/led-base-16384",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_window": [
    1024,
    1024,
    1024,
    1024,
    1024,
    1024
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "ini

In [24]:
untrained_result = evaluate_model(original_model, original_tokenizer)

  0%|          | 0/5 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 1024 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Rouge Results: Score(precision=0.11066118559735125, recall=0.09008842141372886, fmeasure=0.09651794680542444)


And here are the results from the fine-tuned model.

In [25]:
result = evaluate_model(led, tokenizer)

  0%|          | 0/5 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 1024 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Rouge Results: Score(precision=0.18637981057966052, recall=0.11727302917050414, fmeasure=0.13646158246649756)


Let's compare an example abstract from the test dataset to the abstracts predicted by the fine-tuned and untrained models.

Here is the original abstract:

In [26]:
index = random.randint(0, len(result))
dataset['test'][index]['introduction']

'Landslide risk assessment is a critical technical tool for forecasting the occurrence of deadly landslides. However, a reliable model for predicting landslide risk still faces challenging issues. First, the identification and modeling of landslide hazard triggering factors are important technical difficulties. Second, landslide early warning is hampered by ambiguity in the temporal and geographical forecasting of landslide risk. Third, the quantitative evaluation of the landslide hazard triggering factors cannot be applied to actual sites and still requires in-depth study. The goal in the landslide research field is to identify suitable assessment metrics and build accurate models to predict landslide risk. Multisensor analysis still remains difficult. Aiming to address the issues of communication in the Internet of Things (IoT), Joshi et al. presented a useful approach using edge computing to process and analyze landslide data [1]. Li et al. presented a quantitative model of landslid

Here is the abstract predicted by the untrained model:

In [27]:
untrained_result[index]['predicted_introduction']

', compared to that of other models, the data is distributed in a linear time series. The data is divided into two categories: hard attention and soft-attention. The hard attention is a continuous distribution of information, which is the output of the data. The soft attention is an output of information. The output of data is the input of information and the output is the data of the input. The input is the information of the information contained in the data, which can be divided into different types of data, and the data are distributed in different ways. The information contained within the data can be distributed in multiple ways, which are different from the data contained in different types. The landslide instability margins (LIMs) are calculated using (1)–(6), and the calculation process is shown in Figure 3. The first step is to set the four landslide influencing factors as the initialization matrix and then normalize them. Second, the weights are assigned to different landsli

And lastly, here is the abstract predicted by the fine-tuned model:

In [28]:
result[index]['predicted_introduction']

'Landslide risk assessment is an important component of the landslide research field. For the problem of landslide risk assessment, landslide risk is a complex problem, where a large amount of information is collected and stored in a small amount of data. The information collected in the landslide is then used to predict the landslide occurrence, which is easy to obtain directly from sensor data. By using the attention-based temporal convolutional network (TCN) method in landslide risk prediction, the landslide risk can be accurately predicted by using the landslide impact factors (LIF) detemination and the data normalization process (DNP) [1]. In this work, we propose a method that utilizes the attention mechanism and the recurrent neural network (RNN) to predict landslide risk. The RNN is a deep learning time-series prediction method, and it can be used for landslide risk estimation. In this paper, we introduce two landslide risk detection methods, namely the attention mechanisms, wh